# Resonance Mode Energy Transfer (RMET) Model

This notebook presents the **Resonance Mode Energy Transfer (RMET)** model, which proposes that mass and particle-like behavior emerge from resonance modes in a discretized fundamental field, as detailed in the accompanying LaTeX document.

We will explore:
1. The discretized 3D field setup
2. Stiffness operators and resonance modes
3. Minimalist formulation (massless nodes, \( M = I \))
4. Dispersion relations
5. Mode-aware source functionals and Green's operator
6. Emergent mass calculations
7. Field evolution visualization
8. Holonomy calculations for fundamental and composite modes
9. Numerical recipe and interpretation


## 1. Fundamental Field Setup

We define a discretized 3D cubic lattice with \( N_x \times N_y \times N_z \) nodes, representing the fundamental field at the Planck scale. Each node carries a scalar field degree of freedom \( u_i(t) \in \mathbb{R} \), and connectivity is represented by a stiffness matrix \( K = \alpha L \), where \( L \) is the 7-point discrete Laplacian.

The minimalist formulation sets the mass matrix to the identity: \( M = I \).


In [ ]:
import numpy as np

def build_3d_stiffness(Nx, Ny, Nz, a=1.0, alpha=1.0):
    """Build 3D cubic lattice stiffness matrix K = alpha * L."""
    N = Nx * Ny * Nz
    K = np.zeros((N, N))
    
    for i in range(Nx):
        for j in range(Ny):
            for k in range(Nz):
                idx = i + j * Nx + k * Nx * Ny
                # Nearest-neighbor couplings in x, y, z directions
                K[idx, idx] = 6 * alpha / a**2  # Diagonal (3D Laplacian)
                if i > 0:
                    K[idx, idx - 1] = -alpha / a**2
                if i < Nx - 1:
                    K[idx, idx + 1] = -alpha / a**2
                if j > 0:
                    K[idx, idx - Nx] = -alpha / a**2
                if j < Ny - 1:
                    K[idx, idx + Nx] = -alpha / a**2
                if k > 0:
                    K[idx, idx - Nx * Ny] = -alpha / a**2
                if k < Nz - 1:
                    K[idx, idx + Nx * Ny] = -alpha / a**2
    return K

# Example: 3x3x3 lattice
Nx, Ny, Nz = 3, 3, 3
a = 1.0  # Lattice spacing
alpha = 1.0  # Stiffness scaling
K_3d = build_3d_stiffness(Nx, Ny, Nz, a, alpha)
print("3D Stiffness Matrix Shape:", K_3d.shape)


## 2. Resonance Modes

The eigenmodes of the stiffness matrix \( K \) represent the natural resonance excitations of the lattice. The eigenvalue equation is:

\[ K \psi_n = \omega_n^2 \psi_n \]

We compute the eigenmodes and visualize the frequency spectrum.


In [ ]:
from scipy.linalg import eigh
import matplotlib.pyplot as plt

eigvals, eigvecs = eigh(K_3d)
frequencies = np.sqrt(np.abs(eigvals))

plt.plot(frequencies, 'o-')
plt.ylabel(r"Resonant Frequency $\omega_n$")
plt.xlabel("Mode index")
plt.title("Resonance Spectrum of 3D Lattice")
plt.grid(True)
plt.show()


## 3. Dispersion Relation

The dispersion relation for the scalar-isotropic model is given by:

\[ \omega^2(\mathbf{k}) = \alpha \hat{L}(\mathbf{k}), \quad \hat{L}(\mathbf{k}) = \frac{2}{a^2} [3 - \cos(k_x a) - \cos(k_y a) - \cos(k_z a)] \]

For low \( k \), \( \omega(\mathbf{k}) \approx \sqrt{\alpha} |\mathbf{k}| \), with propagation speed \( c = \sqrt{\alpha} \).


In [ ]:
def dispersion_relation(kx, ky, kz, a=1.0, alpha=1.0):
    """Compute dispersion relation for 3D cubic lattice."""
    L_hat = (2 / a**2) * (3 - np.cos(kx * a) - np.cos(ky * a) - np.cos(kz * a))
    return np.sqrt(alpha * L_hat)

# Generate k-points along a path (e.g., [0,0,0] to [pi/a,0,0])
k_max = np.pi / a
k_points = np.linspace(0, k_max, 100)
omega = dispersion_relation(k_points, 0, 0, a, alpha)

# Plot
plt.plot(k_points, omega, label=r'$\omega(k_x, 0, 0)$')
plt.xlabel(r'$k_x$')
plt.ylabel(r'$\omega$')
plt.title('Dispersion Relation for 3D Lattice')
plt.legend()
plt.grid(True)
plt.show()


## 4. Object-Oriented RMET Model

We treat resonance patterns as **objects** in an object-oriented programming (OOP) framework, packaging data (e.g., modal amplitudes, eigenmodes) and behaviors (e.g., propagation, interaction, holonomy).

- **Base Class (`ResonanceModeObject`)**: Defines core attributes (nodes, eigenmodes, amplitudes, frequencies) and methods (propagate, compute source, Green's operator, holonomy).
- **Subclasses**: `PhotonLikeMode` (massless dispersion) and `MassiveMode` (effective inertia from multi-node coupling).


In [ ]:
import numpy as np

class ResonanceModeObject:
    def __init__(self, nodes, K, amplitudes=None, lifetime=np.inf):
        self.nodes = nodes
        self.K = K
        # Compute eigenmodes and frequencies
        eigvals, eigvecs = np.linalg.eigh(K)
        self.frequencies = np.sqrt(np.abs(eigvals))
        self.eigenmodes = eigvecs
        self.amplitudes = np.array(amplitudes) if amplitudes is not None else np.zeros(len(eigvals), dtype=complex)
        self.lifetime = lifetime
        self.field = np.zeros(nodes, dtype=float)

    def set_initial_amplitudes(self, mode_indices, values):
        """Set initial modal amplitudes for specific modes."""
        for idx, val in zip(mode_indices, values):
            self.amplitudes[idx] = val
        return self

    def propagate(self, time_step=1.0):
        """Evolve modal amplitudes with phase e^(-i ω_n t) and compute field u(t)."""
        phases = np.exp(-1j * self.frequencies * time_step)
        self.amplitudes = self.amplitudes * phases
        self.field = np.real(np.dot(self.eigenmodes, self.amplitudes))
        return self

    def compute_source(self, Xi_pq=None, phi_pq=None):
        """Compute mode-aware source J[A](x,t)."""
        J = np.dot(self.eigenmodes, self.amplitudes)  # Linear term
        if Xi_pq is not None and phi_pq is not None:
            # Quadratic term: sum_pq A_p^* A_q Xi_pq phi_pq
            quad_term = np.zeros(self.nodes, dtype=complex)
            for p in range(len(self.amplitudes)):
                for q in range(len(self.amplitudes)):
                    quad_term += np.conj(self.amplitudes[p]) * self.amplitudes[q] * Xi_pq[p, q] * phi_pq[p, q]
            J += quad_term
        return np.real(J)

    def greens_operator(self, omega, Sigma=None):
        """Compute retarded Green's operator G(ω)."""
        if Sigma is None:
            Sigma = np.zeros_like(self.K)
        return np.linalg.inv(self.K - omega**2 * np.eye(self.nodes) + Sigma)

    def medium_response(self, omega, J):
        """Compute medium displacement W(ω) = G(ω) J."""
        G = self.greens_operator(omega)
        return np.dot(G, J)

    def discrete_gradient_operator(self, Nx, Ny, Nz, a=1.0):
        """Build discrete gradient operator D for 3D lattice."""
        N = Nx * Ny * Nz
        D = np.zeros((3 * N, N))
        for i in range(Nx):
            for j in range(Ny):
                for k in range(Nz):
                    idx = i + j * Nx + k * Nx * Ny
                    if i < Nx - 1:
                        D[idx, idx] = -1 / a
                        D[idx, idx + 1] = 1 / a
                    if j < Ny - 1:
                        D[N + idx, idx] = -1 / a
                        D[N + idx, idx + Nx] = 1 / a
                    if k < Nz - 1:
                        D[2 * N + idx, idx] = -1 / a
                        D[2 * N + idx, idx + Nx * Ny] = 1 / a
        return D

    def effective_mass_gradient(self, Nx, Ny, Nz, a=1.0):
        """Compute m_eff using gradient-based method."""
        D = self.discrete_gradient_operator(Nx, Ny, Nz, a)
        Psi_comp = np.dot(self.eigenmodes, self.amplitudes)
        return np.dot(Psi_comp, np.dot(D.T @ D, Psi_comp))

    def compute_berry_connection(self, theta, mode_indices, Nx, Ny, Nz, a=1.0, alpha=1.0):
        """Compute Berry connection A_ij = <ψ_i | ∇_θ | ψ_j> for selected modes."""
        delta_theta = 1e-4
        N_modes = len(mode_indices)
        A = np.zeros((N_modes, N_modes), dtype=complex)
        # Rotate K by theta (simplified: rotate coupling terms in x-direction)
        K_theta = build_3d_stiffness(Nx, Ny, Nz, a, alpha)
        # Perturb K by rotating coupling terms (example: rotate x-direction couplings)
        K_theta_plus = build_3d_stiffness(Nx, Ny, Nz, a, alpha * np.cos(theta + delta_theta))
        eigvals, eigvecs = np.linalg.eigh(K_theta)
        eigvals_plus, eigvecs_plus = np.linalg.eigh(K_theta_plus)
        for i_idx, i in enumerate(mode_indices):
            for j_idx, j in enumerate(mode_indices):
                psi_i = eigvecs[:, i]
                psi_j_plus = eigvecs_plus[:, j]
                A[i_idx, j_idx] = np.dot(psi_i.conj(), (psi_j_plus - psi_i) / delta_theta)
        return A

    def compute_holonomy(self, mode_indices, Nx, Ny, Nz, a=1.0, alpha=1.0):
        """Compute holonomy D(2π) by integrating Berry connection over θ = 0 to 2π."""
        theta_points = np.linspace(0, 2 * np.pi, 100)
        D = np.eye(len(mode_indices), dtype=complex)
        for theta in theta_points[1:]:
            A = self.compute_berry_connection(theta, mode_indices, Nx, Ny, Nz, a, alpha)
            D = D @ (np.eye(len(mode_indices)) + 1j * A * (theta_points[1] - theta_points[0]))
        return D

    def plot_field_evolution(self, timesteps):
        """Plot evolution of field u(t) over time."""
        fields = []
        for t in timesteps:
            self.propagate(time_step=t)
            fields.append(self.field)
        
        plt.figure(figsize=(10, 6))
        for i, field in enumerate(fields):
            plt.plot(field, label=f"t={timesteps[i]:.2f}")
        plt.xlabel("Node Index")
        plt.ylabel(r"Field $u(t)$")
        plt.title("Field Evolution Over Time")
        plt.legend()
        plt.grid(True)
        plt.show()

class PhotonLikeMode(ResonanceModeObject):
    def __init__(self, frequency, amplitude=1.0, nodes=1):
        K = np.array([[frequency**2]])  # Single-mode stiffness
        super().__init__(nodes=nodes, K=K, amplitudes=[amplitude])
    
    def propagate(self, time_step=1.0, c=1.0):
        """Phase evolution for massless dispersion."""
        self.amplitudes *= np.exp(-1j * self.frequencies * time_step / c)
        self.field = np.real(np.dot(self.eigenmodes, self.amplitudes))
        return self

class MassiveMode(ResonanceModeObject):
    def __init__(self, mass, frequency, amplitude=1.0, nodes=10):
        K = np.array([[frequency**2]])  # Simplified for single mode
        super().__init__(nodes=nodes, K=K, amplitudes=[amplitude])
        self.mass = mass
    
    def propagate(self, time_step=1.0):
        """Phase evolution slowed by effective mass."""
        self.amplitudes *= np.exp(-1j * self.frequencies * time_step / (1 + self.mass))
        self.field = np.real(np.dot(self.eigenmodes, self.amplitudes))
        return self

# Example usage
K = build_3d_stiffness(Nx=3, Ny=3, Nz=3)
mode = ResonanceModeObject(nodes=Nx*Ny*Nz, K=K)
mode.set_initial_amplitudes(mode_indices=[0, 1], values=[1.0+0j, 0.5+0j])
mode.propagate(time_step=0.1)
print("Field u(t):", mode.field)


## 5. Mode-Aware Source Functional

The mode-aware source is defined as:

\[ J[\mathbf{A}](x,t) = \sum_n A_n(t) \psi_n(x) + \sum_{p,q} (\mathbf{A}^\dagger \Xi^{(pq)}(x) \mathbf{A}) \varphi_{pq}(x) \]

We compute this source for a simple diagonal \( \Xi^{(pq)} \) and local \( \varphi_{pq} = \psi_p \psi_q \).


In [ ]:
# Compute source J[A]
Xi_pq = np.eye(len(mode.amplitudes))  # Diagonal weight matrix
phi_pq = np.array([mode.eigenmodes[:, p] * mode.eigenmodes[:, q] for p in range(len(mode.amplitudes)) for q in range(len(mode.amplitudes))])
J = mode.compute_source(Xi_pq=Xi_pq, phi_pq=phi_pq.reshape(-1, mode.nodes))
print("Source J[A]:", J)


## 6. Green's Operator and Medium Response

The medium displacement \( W(\omega) \) is computed as:

\[ (K - \omega^2 I + \Sigma(\omega)) W(\omega) = J[\mathbf{A}](\omega) \]

The retarded Green's operator is:

\[ G(\omega) = (K - \omega^2 I + \Sigma(\omega))^{-1} \]


In [ ]:
# Compute medium response W(ω)
omega_bar = 1.0
W = mode.medium_response(omega_bar, J)
print("Medium Displacement W(ω):", W)


## 7. Emergent Mass

The effective mass \( m_{\text{eff}} \) is computed using the gradient-based approach:

\[ m_{\text{eff}} = \Psi_{\text{comp}}^\top D^\top D \Psi_{\text{comp}} \]

where \( D \) is the discrete gradient operator and \( \Psi_{\text{comp}} = \sum_p A_p \psi_p \).


In [ ]:
# Compute effective mass
m_eff = mode.effective_mass_gradient(Nx, Ny, Nz, a)
print("Effective Mass (Gradient):", m_eff)


## 8. Holonomy Calculations

Holonomy calculations assess projective mode-space rotations for degenerate doublets, aiming to verify \( D(2\pi) = -I \). The Berry connection is computed as:

\[ A_{ij}(\theta) = \langle \psi_i(\theta) | \nabla_\theta | \psi_j(\theta) \rangle \]

The holonomy is:

\[ D(2\pi) = \mathcal{P} \exp \left( i \oint A(\theta) \, d\theta \right) \]

We compute this for a doublet or composite mode subspace.


In [ ]:
# Compute holonomy for a doublet (modes 0 and 1)
D_2pi = mode.compute_holonomy(mode_indices=[0, 1], Nx=Nx, Ny=Ny, Nz=Nz, a=a, alpha=alpha)
print("Holonomy D(2π):", D_2pi)

# Check if D(2π) ≈ -I
I_neg = -np.eye(2)
print("Is D(2π) ≈ -I?", np.allclose(D_2pi, I_neg, atol=1e-2))


## 9. Field Evolution Visualization

We visualize the time evolution of the field \( \mathbf{u}(t) = \sum_{n=1}^{N_m} \Re\{A_n(t) \psi_n e^{-i \omega_n t}\} \).


In [ ]:
# Visualize field evolution
timesteps = np.linspace(0, 1.0, 5)
mode.plot_field_evolution(timesteps)


## 10. Numerical Recipe for Minimalist RMET

Following the LaTeX document's recipe:
1. **Assemble K**: Implemented in `build_3d_stiffness`.
2. **Solve eigenproblem**: Included in `ResonanceModeObject.__init__`.
3. **Construct J[A]**: Implemented in `compute_source`.
4. **Compute G(ω)**: Implemented in `greens_operator`.
5. **Solve (W, A) system**: Partially implemented in `medium_response` and `propagate`.
6. **Construct composites and compute m_eff**: Implemented in `effective_mass_gradient`.
7. **Measure holonomy**: Implemented in `compute_holonomy`.
8. **Validate convergence**: To be added (e.g., test with larger lattices or PML boundaries).


## 11. Interpretation of Minimalist RMET

- **Uniform Inertia**: Setting \( M = I \) simplifies dynamics, with emergent mass arising from mode structure and spatial gradients.
- **Physical Behavior**: The model captures dispersion, polarization, composite binding, and projective rotations without explicit node mass.
- **Holonomy**: Non-trivial holonomy (\( D(2\pi) = -I \)) indicates spinor-like behavior, suggesting emergent quantum-like properties.
- **Limitations**: Quantum statistics require additional mechanisms (e.g., topological or quantization approaches).


## 12. Exporting to LaTeX

To generate a LaTeX document from this notebook:
```bash
jupyter nbconvert --to latex RMET_Model_Notebook_OOP.ipynb
```
Compile the resulting `.tex` file with `pdflatex`. Ensure equations use `\begin{equation}...\end{equation}` for proper numbering during export.


## 13. Summary and Next Steps

- Defined a 3D discretized field and stiffness operator.
- Derived resonance modes, dispersion relations, and holonomy.
- Implemented minimalist formulation with \( M = I \).
- Computed mode-aware sources, Green's operator, emergent mass, and holonomy.
- Visualized field evolution.

**Next steps**:
- Implement nonlinear couplings and higher-order resonances.
- Refine holonomy calculations with realistic K(θ) parameterizations.
- Explore mapping to Standard Model structures.
- Validate with larger lattices and boundary conditions.
